Teste para extrair via yahoo finance

In [ ]:
!pip install --upgrade yfinance

  Using cached yfinance-0.2.62-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached frozendict-2.4.6-py312-none-any.whl.metadata (23 kB)
  Using cached curl_cffi-0.11.3-cp39-abi3-win_amd64.whl.metadata (15 kB)
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached yfinance-0.2.62-py2.py3-none-any.whl (118 kB)
Using cached curl_cffi-0.11.3-cp39-abi3-win_amd64.whl (1.4 MB)
Using cached frozendict-2.4.6-py312-none-any.whl (16 kB)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:\\Python312\\Scripts\\sample.exe' -> 'C:\\Python312\\Scripts\\sample.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import os
import ssl
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import certifi
import warnings

# Suprime o FutureWarning de auto_adjust
warnings.filterwarnings("ignore", category=FutureWarning)

# Força uso do certificado correto
os.environ['SSL_CERT_FILE'] = certifi.where()
ssl._create_default_https_context = ssl._create_unverified_context

# Lista de ativos brasileiros
BR_TICKERS = [
    'ALOS3', 'ABEV3', 'ASAI3', 'AURE3', 'AMOB3', 'AZUL4', 'AZZA3', 'B3SA3', 'BBSE3',
    'BBDC3', 'BBDC4', 'BRAP4', 'BBAS3', 'BRKM5', 'BRAV3', 'BRFS3', 'BPAC11', 'CXSE3',
    'CRFB3', 'CCRO3', 'CMIG4', 'COGN3', 'CPLE6', 'CSAN3', 'CPFE3', 'CMIN3', 'CVCB3',
    'CYRE3', 'ELET3', 'ELET6', 'EMBR3', 'ENGI11', 'ENEV3', 'EGIE3', 'EQTL3', 'FLRY3',
    'GGBR4', 'GOAU4', 'NTCO3', 'HAPV3', 'HYPE3', 'IGTI11', 'IRBR3', 'ISAE4', 'ITSA4',
    'ITUB4', 'JBSS3', 'KLBN11', 'RENT3', 'LREN3', 'LWSA3', 'MGLU3', 'POMO4', 'MRFG3',
    'BEEF3', 'MRVE3', 'MULT3', 'PCAR3', 'PETR3', 'PETR4', 'RECV3', 'PRIO3', 'PETZ3',
    'PSSA3', 'RADL3', 'RAIZ4', 'RDOR3', 'RAIL3', 'SBSP3', 'SANB11', 'STBP3', 'SMTO3',
    'CSNA3', 'SLCE3', 'SUZB3', 'TAEE11', 'VIVT3', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5',
    'VALE3', 'VAMO3', 'VBBR3', 'VIVA3', 'WEGE3', 'YDUQ3'
]

def atualizar_cotacoes(file_path):
    # Carrega CSV
    tabela = pd.read_csv(file_path)

    if 'Data' not in tabela.columns:
        raise ValueError("O arquivo CSV não possui uma coluna chamada 'Data'")

    # Usa data fixa como fallback
    data_ultima = "02/01/2018 16:56:00"
    try:
        data_datetime = datetime.strptime(data_ultima.split()[0], '%d/%m/%Y')
    except:
        data_datetime = datetime.now() - timedelta(days=30)

    data_inicio = (data_datetime + timedelta(days=1)).strftime('%Y-%m-%d')
    data_fim = datetime.now().strftime('%Y-%m-%d')

    colunas_ativos = tabela.columns[1:]
    ativos_formatados = [
        f"{ativo}.SA" if ativo in BR_TICKERS else ativo
        for ativo in colunas_ativos
    ]

    batch_size = 50
    dados_novos = pd.DataFrame()

    for i in range(0, len(ativos_formatados), batch_size):
        batch = ativos_formatados[i:i + batch_size]
        try:
            dados_batch = yf.download(
                batch,
                start=data_inicio,
                end=data_fim,
                interval="1d",
                auto_adjust=False,
                progress=False
            )['Adj Close']

            if isinstance(dados_batch, pd.Series):
                dados_batch = dados_batch.to_frame()

            dados_novos = pd.concat([dados_novos, dados_batch], axis=1)

        except Exception as e:
            print(f"Erro ao baixar lote {i//batch_size + 1}: {e}")
            continue

    if not dados_novos.empty:
        dados_novos.columns = [col.replace('.SA', '') for col in dados_novos.columns]
        dados_novos.index = dados_novos.index.strftime('%d/%m/%Y %H:%M:%S')
        dados_novos = dados_novos.ffill().bfill()

        for col in dados_novos.columns:
            dados_novos[col] = dados_novos[col].apply(
                lambda x: f"{x:.2f}".replace('.', ',') if pd.notnull(x) else ""
            )

        dados_novos = dados_novos.reset_index().rename(columns={'Date': 'Data'})
        tabela_atualizada = pd.concat([tabela, dados_novos], ignore_index=True)
        tabela_atualizada.to_csv(file_path, index=False, sep=',', header=True, quoting=0)
        print("Cotações atualizadas com sucesso!")
        return True
    else:
        print("Nenhum dado novo foi encontrado.")
        return False

if __name__ == "__main__":
    file_path = 'arquivos/Dados_Ativos_B3_AdjClose.csv'
    sucesso = atualizar_cotacoes(file_path)
    print("Arquivo atualizado com sucesso!" if sucesso else "Houve um problema na atualização.")


Cotações atualizadas com sucesso!
Arquivo atualizado com sucesso!


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    
    tickers = []
    for row in table.findAll('tr')[1:]:  # Pula o cabeçalho
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)
    
    return tickers

# Obter os tickers
sp500_tickers = get_sp500_tickers()

# Criar a linha de vírgulas (uma vírgula a menos que o número de tickers)
linha_virgulas = [',' * (len(sp500_tickers) - 1)]

# Juntar os tickers e a linha de vírgulas
dados_para_csv = [sp500_tickers, linha_virgulas]

# Criar DataFrame e salvar
df = pd.DataFrame(dados_para_csv)
df.to_csv('sp500_tickers_com_virgulas.csv', index=False, header=False)

print(f"✅ Arquivo criado com {len(sp500_tickers)} tickers + linha de vírgulas!")
print("Primeira linha:", sp500_tickers[:3], "...")
print("Segunda linha:", linha_virgulas[0][:30] + "...")  # Mostra apenas parte das vírgulas

✅ Arquivo criado com 503 tickers + linha de vírgulas!
Primeira linha: ['MMM', 'AOS', 'ABT'] ...
Segunda linha: ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...


: 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

# Lista completa de ativos brasileiros (precisa de .SA)
BR_TICKERS = [
    'ALOS3', 'ABEV3', 'ASAI3', 'AURE3', 'AMOB3', 'AZUL4', 'AZZA3', 'B3SA3', 'BBSE3', 
    'BBDC3', 'BBDC4', 'BRAP4', 'BBAS3', 'BRKM5', 'BRAV3', 'BRFS3', 'BPAC11', 'CXSE3', 
    'CRFB3', 'CCRO3', 'CMIG4', 'COGN3', 'CPLE6', 'CSAN3', 'CPFE3', 'CMIN3', 'CVCB3', 
    'CYRE3', 'ELET3', 'ELET6', 'EMBR3', 'ENGI11', 'ENEV3', 'EGIE3', 'EQTL3', 'FLRY3', 
    'GGBR4', 'GOAU4', 'NTCO3', 'HAPV3', 'HYPE3', 'IGTI11', 'IRBR3', 'ISAE4', 'ITSA4', 
    'ITUB4', 'JBSS3', 'KLBN11', 'RENT3', 'LREN3', 'LWSA3', 'MGLU3', 'POMO4', 'MRFG3', 
    'BEEF3', 'MRVE3', 'MULT3', 'PCAR3', 'PETR3', 'PETR4', 'RECV3', 'PRIO3', 'PETZ3', 
    'PSSA3', 'RADL3', 'RAIZ4', 'RDOR3', 'RAIL3', 'SBSP3', 'SANB11', 'STBP3', 'SMTO3', 
    'CSNA3', 'SLCE3', 'SUZB3', 'TAEE11', 'VIVT3', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5', 
    'VALE3', 'VAMO3', 'VBBR3', 'VIVA3', 'WEGE3', 'YDUQ3'
]

def atualizar_cotacoes(file_path):
    # Carregar o arquivo CSV
    tabela = pd.read_csv(file_path)
    
    # Verificar coluna Data
    if 'Data' not in tabela.columns:
        raise ValueError("O arquivo CSV não possui uma coluna chamada 'Data'")
    
    # Extrair dados
    ultima_linha = tabela.iloc[-1]
    data_ultima = "02/01/2018 16:56:00"
    colunas_ativos = tabela.columns[1:]  # Excluir a coluna Data
    
    # Preparar tickers formatados
    ativos_formatados = []
    for ativo in colunas_ativos:
        if ativo in BR_TICKERS:
            ativos_formatados.append(ativo)  # Adiciona .SA para brasileiros
        else:
            ativos_formatados.append(ativo)  # Mantém original para S&P 500
    
    # Converter data
    try:
        data_datetime = datetime.strptime(data_ultima.split()[0], '%d/%m/%Y')
        data_inicio = (data_datetime + timedelta(days=1)).strftime('%Y-%m-%d')
    except:
        # Caso a data já esteja em formato diferente
        data_inicio = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    data_fim = datetime.now().strftime('%Y-%m-%d')
    
    # Baixar cotações em lotes
    batch_size = 10  # Yahoo Finance tem limite de tickers por requisição
    dados_novos = pd.DataFrame()
    
    for i in range(0, len(ativos_formatados), batch_size):
        batch = ativos_formatados[i:i + batch_size]
        try:
            dados_batch = yf.download(
                batch,
                start=data_inicio,
                end=data_fim,
                interval="1d",
            )['Adj Close']
            
            # Corrigir formato quando há apenas 1 ativo no batch
            if len(batch) == 1:
                dados_batch = pd.DataFrame(dados_batch)
                dados_batch.columns = [batch[0]]
            
            dados_novos = pd.concat([dados_novos, dados_batch], axis=1)
        except Exception as e:
            print(f"Erro ao baixar lote {i//batch_size + 1}: {e}")
            continue
    
    # Processar os dados
    if not dados_novos.empty:
        # Remover sufixos .SA dos nomes das colunas
        dados_novos.columns = [col.replace('.SA', '') for col in dados_novos.columns]
        
        # Formatar datas
        dados_novos.index = dados_novos.index.strftime('%d/%m/%Y %H:%M:%S')
        
        # Preencher valores ausentes
        dados_novos = dados_novos.ffill().bfill()
        
        # Formatar números com 2 casas decimais e vírgula
        for col in dados_novos.columns:
            dados_novos[col] = dados_novos[col].apply(
                lambda x: f"{x:.2f}".replace('.', ',') if pd.notnull(x) else ""
            )
        
        # Resetar índice e renomear coluna de data
        dados_novos = dados_novos.reset_index().rename(columns={'Date': 'Data'})
        
        # Concatenar com dados originais
        tabela_atualizada = pd.concat([tabela, dados_novos], ignore_index=True)
        
        # Salvar arquivo
        tabela_atualizada.to_csv(file_path, index=False, sep=',', header=True, quoting=0)
        print("Cotações atualizadas com sucesso!")
        return True
    else:
        print("Nenhum dado novo foi encontrado.")
        return False

# Uso do código
if __name__ == "__main__":
    file_path = 'arquivos/Dados_Ativos_B3.csv'
    sucesso = atualizar_cotacoes(file_path)
    
    if sucesso:
        print("Arquivo atualizado com sucesso!")
    else:
        print("Houve um problema na atualização.")

[**********************50%                       ]  5 of 10 completedHTTP Error 404: 
HTTP Error 404: 
[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['ABEV3', 'BBSE3', 'ALOS3', 'AZZA3', 'AMOB3', 'AURE3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBDC3']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['BRAP4', 'CCRO3', 'BBDC4', 'BRKM5', 'BRAV3', 'BBAS3', 'BPAC11', 'CXSE3', 'BRFS3', 'CRFB3']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['ELET6', 'CMIN3', 'CSAN3', 'ELET3', 'CYRE3', 'CPFE3', 'CMIG4', 'COGN3', 'CVCB3', 'CPLE6']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  10 of 10 completed

10 Failed downloads:
['HAPV3', 'EMBR3', 'EGIE3', 'GOAU4', 'GGBR4', 'NTCO3', 'ENEV3', 'ENGI11', 'FL

Nenhum dado novo foi encontrado.
Houve um problema na atualização.


In [10]:
# Número total de ativos (503)
total_ativos = 503

# Número de vírgulas por linha (502, ou seja, uma a menos que o total de ativos)
virgulas_por_linha = total_ativos - 1

# Cria 503 linhas, cada uma com 502 vírgulas
linhas = [',' * virgulas_por_linha for _ in range(total_ativos)]

# Salva em um arquivo CSV
with open('503_linhas_virgulas.csv', 'w') as f:
    f.write('\n'.join(linhas))

print(f"✅ Arquivo criado com {total_ativos} linhas de {virgulas_por_linha} vírgulas cada!")

✅ Arquivo criado com 503 linhas de 502 vírgulas cada!


In [11]:
!pip install --upgrade selenium
!pip install --upgrade webdriver-manager


  Using cached selenium-4.33.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
Using cached selenium-4.33.0-py3-none-any.whl (9.4 MB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:\\Python312\\Scripts\\wsdump.exe' -> 'C:\\Python312\\Scripts\\wsdump.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
  Attempting uninstall: webdriver-manager
    Found existing installation: webdriver-manager 4.0.1
    Uninstalling webdriver-manager-4.0.1:
      Successfully uninstalled webdriver-manager-4.0.1



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Testes de extrair usando selenium (raspagem de tela)

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import pandas as pd

# Carregar o arquivo CSV
file_path = 'csv/TesteTotal2.csv'
data = pd.read_csv(file_path)

# Obter os nomes dos ativos a partir das colunas do arquivo CSV, excluindo 'Data'
ativos = [col for col in data.columns if col != 'Data']

# Configuração do WebDriver com modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ativa o modo headless
chrome_options.add_argument("--disable-gpu")  # Desabilita a GPU (opcional)
chrome_options.add_argument("--window-size=1920,1080")  # Define o tamanho da janela (opcional)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Dicionário para armazenar os preços dos ativos
precos = {}

# XPath para a barra de pesquisa na página inicial
search_box_xpath = '//*[@id="yDmH0d"]/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]'

try:
    # Loop através dos ativos
    for ativo in ativos:
        # Abrir o site do Google Finance
        driver.get("https://www.google.com/finance/")

        # Tentar encontrar a barra de pesquisa usando o XPath
        try:
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, search_box_xpath))
            )
        except:
            raise Exception(f"Barra de pesquisa não encontrada para o ativo: {ativo}")

        # Limpar o campo de busca antes de cada uso
        search_box.clear()

        # Inserir o nome do ativo no campo de busca
        search_box.send_keys(ativo)

        # Simular o pressionamento da tecla Enter
        search_box.send_keys(Keys.RETURN)

        # Esperar o valor carregar e obter o preço
        try:
            value_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "P6K39c"))
            )
        except:
            raise Exception(f"Preço do ativo {ativo} não encontrado.")

        # Obter o texto do preço e remover 'R$' e possíveis espaços
        preco_texto = value_element.text.replace('R$', '').strip()
        # Armazenar o preço como string no dicionário
        precos[ativo] = preco_texto
        print(f"{ativo}: {precos[ativo]}")

    # Adicionar a data e os preços no DataFrame
    data_atual = datetime.now()
    data_atual = data_atual.strftime('%d/%m/%Y %H:%M:%S')

    nova_linha = {'Data': data_atual}
    for ativo, preco in precos.items():
        nova_linha[ativo] = preco

    # Adicionar a nova linha ao DataFrame
    data = pd.concat([data, pd.DataFrame([nova_linha])], ignore_index=True)

finally:
    # Fechar o navegador
    driver.quit()

# Salvar o arquivo atualizado
data.to_csv(file_path, index=False)

# Mostrar o DataFrame atualizado
print(data)


FileNotFoundError: [Errno 2] No such file or directory: 'csv/TesteTotal2.csv'

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm'

driver.get(url)

WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "bvmf_iframe")))

planilha = driver.find_element("xpath", '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a')

driver.execute_script("arguments[0].click();", planilha)

time.sleep(5)

driver.quit()

In [ ]:
import pandas as pd

# Lê o arquivo CSV
df = pd.read_csv('IBOVDia_26-03-25.csv', sep=';', encoding='utf-8')

# Extrai a coluna "Aux" e remove as últimas 2 linhas
codigos_ativos = df['Aux'].tolist()[:-2]

# Cria um DataFrame onde cada código é uma coluna
df_output = pd.DataFrame([codigos_ativos])

# Salva como CSV sem cabeçalho e sem índice
df_output.to_csv('codigos_ibovespa.csv', index=False, header=False, encoding='utf-8-sig')

print("✅ Arquivo 'codigos_ibovespa.csv' criado com sucesso!")
print(f"Total de códigos extraídos: {len(codigos_ativos)}")
print("Exemplo dos primeiros códigos:", codigos_ativos[:5])

✅ Arquivo 'codigos_ibovespa.csv' criado com sucesso!
Total de códigos extraídos: 87
Exemplo dos primeiros códigos: ['ALOS3', 'ABEV3', 'ASAI3', 'AURE3', 'AMOB3']


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def selecionar_ativos(data, limite=10):

    """
    Exibe os ativos disponíveis e permite que o usuário selecione até um número limitado de ativos.
    
    Parâmetros:
    - data: DataFrame com os dados dos ativos, deve conter uma coluna 'Data'.
    - limite: número máximo de ativos a serem selecionados (padrão: 10).
    
    Retorna:
    - Lista com os nomes dos ativos selecionados.
    """
    all_assets = [col for col in data.columns if col != 'Data']
    print("Ativos disponíveis:")
    for i, asset in enumerate(all_assets):
        print(f"{i + 1}: {asset}")

    while True:
        selected_indices = input(f"Digite os números dos ativos que deseja usar (máx {limite}), separados por vírgula: ")
        selected_indices = selected_indices.split(',')

        try:
            selected_indices = [int(i.strip()) - 1 for i in selected_indices]
            if len(selected_indices) > limite:
                print(f"Você selecionou mais de {limite} ativos. Por favor, selecione no máximo {limite}.")
                continue
            if any(i < 0 or i >= len(all_assets) for i in selected_indices):
                print("Algum índice está fora do intervalo. Tente novamente.")
                continue
            break
        except ValueError:
            print("Entrada inválida. Digite os números separados por vírgula.")

    asset_columns = [all_assets[i] for i in selected_indices]
    print(f"\nAtivos selecionados: {asset_columns}")
    return asset_columns

# Definir o caminho para os arquivos CSV
file_path = 'arquivos/TesteTotal2.csv'
cdi_file_path = 'arquivos/cdi_data_total.csv'

# Carregar os dados do CDI e ATIVOS do arquivo CSV completo
cdi_data_total = pd.read_csv(cdi_file_path)
data = pd.read_csv(file_path)

# Carregar os dados históricos dos ativos
data['Data'] = pd.to_datetime(data['Data'], format='%d/%m/%Y %H:%M:%S')

# Converter a coluna 'data' para datetime e adicionar o horário "16:56:00"
cdi_data_total['data'] = pd.to_datetime(cdi_data_total['data'], dayfirst=True) + pd.Timedelta(hours=16, minutes=56)

# Remove o caractere de separador de milhar e converte as colunas de ativos para float
asset_columns = selecionar_ativos(data)

for col in asset_columns:
    data[col] = data[col].str.replace(',', '.').astype(float)

# Inicializa variáveis
aporte_inicial = 1000
aporte_mensal = 400
total_investido = 0
quantities = {asset: 0 for asset in asset_columns}

# ----------------------------
# Inicialização de variáveis para o portfólio otimizado
# ----------------------------

n_assets = len(asset_columns)
x_initial = np.ones(n_assets) / n_assets
b = np.ones(n_assets) / n_assets
total_invested_portfolio = 0
quantities_portfolio = {asset: 0 for asset in asset_columns}
# Para cada ativo, manter quantidades separadas
quantities_individual = {asset: 0 for asset in asset_columns}
total_invested_individual = {asset: 0 for asset in asset_columns}

# Listas para armazenar os valores dos portfólios de cada estratégia
portfolio_values_eficiente = []
portfolio_values_paridade = []
individual_portfolio_values = {asset: [] for asset in asset_columns}
monthly_risk_parity_values = []  # Lista para armazenar os valores mensais da Paridade de Risco

# ----------------------------
# Funções Auxiliares
# ----------------------------

def filter_cdi_data(month, df_cdi_total, df_assets):
    one_year_before = month - pd.DateOffset(years=1)
    
    # Filtrar pelo período de 1 ano antes até o mês atual
    cdi_filtered = df_cdi_total[(df_cdi_total['data'] >= one_year_before) & (cdi_data_total['data'] < month)]

    # Filtrar para garantir que as datas do CDI correspondam às do arquivo de ativos
    cdi_filtered = cdi_filtered[cdi_filtered['data'].isin(df_assets['Data'])]

    return cdi_filtered

def get_yearly_data(month, data):
    one_year_before = month - pd.DateOffset(years=1)
    yearly_data = data[(data['Data'] >= one_year_before) & (data['Data'] < month)]
    return yearly_data

def calculate_investment(total_portfolio_value, weights, current_prices, asset_columns):
    quantities = {}
    for i, asset in enumerate(asset_columns):
        current_price = current_prices[asset]
        amount_to_invest_in_asset = total_portfolio_value * weights[i]
        if current_price > 0:
            quantities[asset] = amount_to_invest_in_asset / current_price
        else:
            quantities[asset] = 0
    return quantities

# ------------------------
# Funções de Verificação
# ------------------------

def calculate_risk_contributions(x, cov_matrix):
    sigma_x = np.dot(cov_matrix, x)
    total_risk = np.sqrt(np.dot(x.T, sigma_x))
    marginal_risk = sigma_x / total_risk
    risk_contributions = x * marginal_risk
    return risk_contributions, total_risk

def check_equal_risk_contributions(risk_contributions):
    total_risk = np.sum(risk_contributions)
    n_assets = len(risk_contributions)
    target_risk = total_risk / n_assets
    return np.all(np.isclose(risk_contributions, target_risk, atol=1e-3))

# ----------------------------
# Funções de otimização
# ----------------------------

def calculate_variance(x, cov_matrix):
    return np.dot(x.T, np.dot(cov_matrix, x))

def objective(x, cov_matrix, b):
    variance = calculate_variance(x, cov_matrix)
    sqrt_variance = np.sqrt(variance)
    w = x / sqrt_variance
    quadratic_term = 0.5 * np.dot(w.T, b / w)
    log_term = np.dot(b.T, np.log(w))
    return quadratic_term - log_term

def weight_sum_constraint(x):
    return np.sum(x) - 1.0

def weight_bounds(n_assets):
    return [(1e-19, 1) for _ in range(n_assets)]
81
def solve_system_8(cov_matrix, b, initial_x):
    n_assets = len(b)
    constraints = {'type': 'eq', 'fun': weight_sum_constraint}
    bounds = weight_bounds(n_assets)
    solution = minimize(objective, initial_x, args=(cov_matrix, b),
                        method='SLSQP', bounds=bounds, constraints=constraints, options={'maxiter': 1000, 'ftol': 1e-9})
    if not solution.success:
        raise ValueError(f"Falha na otimização: {solution.message}")
    return solution.x, solution.fun

# ----------------------------
# Estratégias de Investimento
# ----------------------------

def EstrategiaEficiente(aporte_inicial, aporte_mensal, total_investido, portfolio_values_eficiente):
    global quantities  # Para manter as quantidades atualizadas fora da função
    print(2023)
    print(start_2023)
    for month in pd.date_range(start=start_2023, end=end_2023, freq='MS'):

        # Definir a data de investimento como o primeiro dia útil do mês em um horário específico
        investment_date = month + pd.offsets.BMonthBegin(0)
        investment_datetime = investment_date.replace(hour=16, minute=56, second=0)
        one_year_before = investment_date - pd.DateOffset(years=1)

        yearly_data = data[(data['Data'] >= one_year_before) & (data['Data'] < investment_date)]
        yearly_dataaux = data[(data['Data'] >= one_year_before) & (data['Data'] <= investment_datetime)]
        # Filtrar os dados de CDI para o período relevante (um ano antes do mês atual) e garantir que as datas correspondam ao arquivo de ativos
        cdidiario_filtered = filter_cdi_data(month, cdi_data_total, yearly_data)
        
        # Cálculo de retornos diários para todos os ativos
        returns = yearly_data[asset_columns].pct_change().dropna()

        if returns.empty:
            print(f"Erro: Não há retornos suficientes para calcular a matriz de covariância em {month.strftime('%m/%Y')}. Pulando este mês.")
            continue

        # Cálculo da matriz de covariância entre todos os ativos
        cov_matrix = returns.cov()

        # Cálculo das médias dos retornos para todos os ativos
        expected_returns = returns.mean().values

        # Vetor elementar e transposto para múltiplos ativos
        e_transposto = np.ones(len(asset_columns)).reshape(1, -1)

        # Inversa da matriz de covariância
        try:
            cov_matrix_inv = np.linalg.inv(cov_matrix)
        except np.linalg.LinAlgError:
            print(f"Erro: Matriz de covariância singular em {month.strftime('%m/%Y')}. Pulando este mês.")
            continue

        # Calcular a média do retorno do CDI filtrado
        CDIRet = cdidiario_filtered['valor'].mean()

        # Cálculo de Rf_e para múltiplos ativos (usando o retorno do CDI)
        Rf_e = np.full((len(asset_columns), 1), CDIRet / 100)

        # Cálculo de M - Rf_e
        M_Rfe = expected_returns.reshape(-1, 1) - Rf_e

        # Multiplicações necessárias para obter as porcentagens
        V_M_Rfe = np.matmul(cov_matrix_inv, M_Rfe).flatten()
        V_eT = np.matmul(e_transposto, cov_matrix_inv)

        # Certifique-se de que Div seja um escalar
        Div = np.matmul(V_eT, M_Rfe)[0, 0]

        if Div == 0:
            #print(f"Erro: Divisor zero em {month.strftime('%m/%Y')}. Pulando este mês.")
            continue

        # Cálculo das porcentagens para todos os ativos
        percentages = V_M_Rfe / Div

        # Determinar o valor de investimento para este mês
        if month == pd.to_datetime(start_2023):
            amount_to_invest = aporte_inicial
        else:
            amount_to_invest = aporte_mensal
        total_investido += amount_to_invest
        
        # Calcular os preços atuais na data de investimento
        current_prices = yearly_dataaux.iloc[-1][asset_columns].to_dict()

        # Calcular as quantidades a serem investidas em cada ativo com base nas porcentagens otimizadas
        new_quantities = calculate_investment(amount_to_invest, percentages, current_prices, asset_columns)

        # Atualizar as quantidades do portfólio
        for asset in asset_columns:
            quantities[asset] += new_quantities[asset]

        # Definir o início e fim do mês atual
        start_of_month = month
        end_of_month = month + pd.offsets.MonthEnd(0)

        # Filtrar dados diários para o mês atual
        daily_data = data[(data['Data'] >= start_of_month) & (data['Data'] <= end_of_month)].reset_index(drop=True)
        
        if not daily_data.empty:
            # Substituir preços faltantes com o último preço disponível para evitar NaNs
            daily_prices = daily_data[asset_columns].ffill()
            
            # --- Avaliação diária do portfólio otimizado ---
            portfolio_daily_values = daily_prices.multiply(list(quantities.values()), axis=1).sum(axis=1)
            portfolio_values_eficiente.extend(zip(daily_data['Data'], portfolio_daily_values))

def ParidadeDeRisco(aporte_inicial, aporte_mensal, n_assets, x_initial, b, portfolio_values_paridade, individual_portfolio_values):
    global quantities_portfolio, quantities_individual, total_invested_portfolio, monthly_risk_parity_values
    for month in pd.date_range(start=start_2023, end=end_2023, freq='MS'):
        
        # Definir a data de investimento como o primeiro dia útil do mês em um horário específico
        investment_date = month + pd.offsets.BMonthBegin(0)
        investment_datetime = investment_date.replace(hour=16, minute=56, second=0)
        
        # Definir o período de retrospectiva de um ano
        one_year_before = investment_date - pd.DateOffset(years=1)
        
        # Filtrar dados até a data de investimento
        yearly_data = data[(data['Data'] >= one_year_before) & (data['Data'] < investment_date)]
        yearly_dataaux = data[(data['Data'] >= one_year_before) & (data['Data'] <= investment_datetime)]
        # Garantir que haja dados suficientes para o cálculo da covariância
        if len(yearly_data) < 2:
            #print(f"Erro: Não há dados suficientes para calcular a covariância até {investment_date.strftime('%d/%m/%Y')}. Pulando este mês.")
            continue
        # Calcular a matriz de covariância
        cov_matrix = yearly_data[asset_columns].cov() * 1e-2  
        
        # Verificar se a matriz de covariância é singular
        if cov_matrix.isnull().values.any():
            #print(f"Erro: Matriz de covariância contém valores NaN em {month.strftime('%m/%Y')}. Pulando este mês.")
            continue
        
        try:
            cov_matrix_np = cov_matrix.values
            # Otimizar para encontrar os pesos ótimos
            optimal_x, _ = solve_system_8(cov_matrix_np, b, x_initial)
            #print(optimal_x * 1000)
        except ValueError as e:
            #print(e)
            continue
        x_initial = optimal_x  # Atualizar os pesos iniciais para a próxima iteração
        
        # Determinar o valor de investimento para este mês
        
        if month == pd.to_datetime(start_2023):
            amount_to_invest = aporte_inicial
        else:
            amount_to_invest = aporte_mensal
        total_invested_portfolio += amount_to_invest
        
        # Calcular os preços atuais na data de investimento
        current_prices = yearly_dataaux.iloc[-1][asset_columns].to_dict()
        
        # Calcular as quantidades a serem investidas em cada ativo com base nos pesos otimizados
        new_quantities = calculate_investment(amount_to_invest, optimal_x, current_prices, asset_columns)
        #print(new_quantities)
        # Atualizar as quantidades do portfólio
        for asset in asset_columns:
            quantities_portfolio[asset] += new_quantities[asset]
        #print(month)
        
        # Calcular as contribuições de risco
        risk_contributions, total_risk = calculate_risk_contributions(optimal_x, cov_matrix_np)
        
        target_risk = 1 / n_assets
        if check_equal_risk_contributions(risk_contributions):
            pass  # Nenhuma ação necessária
        else:
            pass  # Nenhuma ação necessária
        
        # ----------------------------
        # Atualizações de investimentos individuais
        # ----------------------------
        
        for asset in asset_columns:
            # Alocar todo o investimento para este ativo
            total_invested_individual[asset] += amount_to_invest
            current_price_asset = current_prices[asset]
            #print(amount_to_invest)
            #if asset in ('IVVB11', 'BOVA11'):
            #    print(current_price_asset)  
            if current_price_asset > 0:
                quantities_individual[asset] += amount_to_invest / current_price_asset
            else:
                pass  # Nenhuma ação necessária
        
        # ----------------------------
        # Avaliação diária do portfólio para o mês atual
        # ----------------------------
        
        # Definir o início e fim do mês atual
        start_of_month = month
        end_of_month = month + pd.offsets.MonthEnd(0)
        
        # Filtrar dados diários para o mês atual
        daily_data = data[(data['Data'] >= start_of_month) & (data['Data'] <= end_of_month)].reset_index(drop=True)
        
        if not daily_data.empty:
            # Substituir preços faltantes com o último preço disponível para evitar NaNs
            daily_prices = daily_data[asset_columns].ffill()
            
            # --- Avaliação diária do portfólio otimizado ---
            portfolio_daily_values = daily_prices.multiply(list(quantities_portfolio.values()), axis=1).sum(axis=1)
            portfolio_values_paridade.extend(zip(daily_data['Data'], portfolio_daily_values))
            
            # --- Avaliação diária dos investimentos individuais ---
            for asset in asset_columns:
                individual_daily_values = daily_prices[asset] * quantities_individual[asset]
                individual_portfolio_values[asset].extend(zip(daily_data['Data'], individual_daily_values))
            
            # ----------------------------
            # Registro dos valores no final do mês
            # ----------------------------
            
            # Obter a data e preços do último dia do mês
            end_of_month_date = daily_data['Data'].iloc[-1]
            end_of_month_prices = daily_data.iloc[-1][asset_columns]
            
            # Calcular o valor de cada ativo no portfólio
            asset_values = end_of_month_prices * pd.Series(quantities_portfolio)
            
            # Calcular o valor total do portfólio
            total_portfolio_value = asset_values.sum()
            
            # Calcular as porcentagens de cada ativo no portfólio
            asset_percentages = asset_values / total_portfolio_value
            
            # Armazenar os valores mensais
            monthly_risk_parity_values.append({
                'Date': end_of_month_date,
                'TotalValue': total_portfolio_value,
                'AssetValues': asset_values,
                'AssetPercentages': asset_percentages
            })
        
# ----------------------------
# Definir Período de Análise
# ----------------------------

start_2023 = '2018-01-02'
end_2023 = '2023-12-31'

# ----------------------------
# Executar as Estratégias
# ----------------------------

EstrategiaEficiente(aporte_inicial, aporte_mensal, total_investido, portfolio_values_eficiente)
ParidadeDeRisco(aporte_inicial, aporte_mensal, n_assets, x_initial, b, portfolio_values_paridade, individual_portfolio_values)

# Converte listas em DataFrame para visualização
df_eficiente = pd.DataFrame(portfolio_values_eficiente, columns=['Data', 'Eficiente'])
df_paridade = pd.DataFrame(portfolio_values_paridade, columns=['Data', 'Paridade'])



In [ ]:
import numpy as np

# Matriz de covariância
cov_matrix = np.array([
    [0.000031, 0.000023, 0.000045, 0.000031, 0.000017, 0.000026],
    [0.000023, 0.000183, 0.000166, 0.000065, 0.000062, 0.000143],
    [0.000045, 0.000166, 0.000427, 0.000005, 0.000031, 0.000127],
    [0.000031, 0.000065, 0.000005, 0.000227, 0.000124, 0.000219],
    [0.000017, 0.000062, 0.000031, 0.000124, 0.000212, 0.000246],
    [0.000026, 0.000143, 0.000127, 0.000219, 0.000246, 0.000424]
])

# Verificar se a matriz é inversível
if np.linalg.det(cov_matrix) == 0:
    print("Matriz singular: determinante zero. Não pode ser invertida.")
else:
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    print("Matriz Inversa (Σ⁻¹):")
    print(np.round(inv_cov_matrix, 2))  # Arredondando para 2 casas decimais

Matriz Inversa (Σ⁻¹):
[[ 51128.09   -526.92  -7320.26 -12002.15  -6613.82   9271.62]
 [  -526.92  10633.94  -3319.34   -826.56   1151.35  -2800.97]
 [ -7320.26  -3319.34   5178.03   3694.91   2551.52  -3371.4 ]
 [-12002.15   -826.56   3694.91  12612.68   2915.68  -8298.2 ]
 [ -6613.82   1151.35   2551.52   2915.68  16667.84 -11923.46]
 [  9271.62  -2800.97  -3371.4   -8298.2  -11923.46  14948.4 ]]
